In [1]:
import numpy as np

# Решение системы нелинейных уравнений методом Ньютона 

In [13]:
def f(x):
    return [np.tan(x[0] * x[1] + 0.2) - x[0] ** 2, 0.6 * x[0] ** 2 + 2 * x[1] ** 2 - 1]

In [14]:
def g(x):
    return [[x[1] / np.cos(x[0] * x[1] + 0.2) ** 2 - 2 * x[0], 1 / np.cos(x[0] * x[1] + 0.2) ** 2],
           [1.6 * x[0], 4 * x[1]]]

In [27]:
# n - количество уравнений 
# x - начальное приближение
# f - f(x)
# g - g(x) вычисляет матрицу производных 

def Newts(x, f, g, eps):
    k = 0
    while (True):
        x_ = x - np.linalg.inv(g(x)) @ f(x)
        k += 1
        if np.linalg.norm(x_ - x) < eps:
            return x_, k
        x = x_


In [28]:
x = [1, 1]
g(x)

[[5.615963967207052, 7.615963967207052], [1.6, 4]]

In [29]:
ans = Newts(x, f, g, 1e-6, )
ans

(array([0.87646187, 0.51917663]), 10)

In [31]:
f(ans[0])

[-4.940548370413467e-08, -1.3592064251888303e-08]

# Решение систем линейных уравнений методом Гаусса

In [151]:
A = np.array([[2.16, 1.96, 1.56],
    [3.55, 3.23, 2.78],
    [4.85, 4.47, 3.97]])

In [152]:
b = np.array([[13.16], [21.73], [29.75]])

In [153]:
def gaus(A, b):
    n = A.shape[0]
    for i in range(n):
        b[i] /= A[i][i]
        A[i] /= A[i][i]
        for j in range(i + 1, n):
            b[j] -= A[j][i] * b[i]
            A[j] -= A[j][i] * A[i]
            
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            b[j] -= b[i] * A[j][i]
    return b

In [154]:
ans = gaus(A.copy(), b.copy())
ans

array([[ 6.06990622],
       [-0.35959079],
       [ 0.48320546]])

In [155]:
A @ ans - b

array([[1.77635684e-15],
       [0.00000000e+00],
       [0.00000000e+00]])

# Решение систем уравнений методом простых итераций 

In [119]:
C = np.array([[13.4, 0.581, 0.702, 0.0822],
             [0.0408, 12.5, 0.65, 0.77],
             [0.0356, 0.0477, 11.6, 0.718],
             [0.0304, 0.0425, 0.0546, 10.7]])

In [120]:
d = np.array([[17.7828], [19.0599], [19.9744], [20.5261]])

In [144]:
def simple_iter(C, d, x, eps):
    k = 0
    n = C.shape[0]
    while(True):
        x_ = np.zeros(n)
        
        for i in range(n):
            x_[i] = d[i] / C[i][i]
            for j in range(n):
                if i != j:
                    x_[i] -= C[i][j] / C[i][i] * x[j]
                    
        k += 1
        if np.linalg.norm(x_ - x) < eps:
            return [x_, k]
        x = x_

In [145]:
x = np.array([1, 1, 1, 1])

In [148]:
ans = simple_iter(C, d, x, 1e-15)
ans[0] = ans[0].reshape(-1, 1)
ans

[array([[1.17457001],
        [1.32086943],
        [1.59519211],
        [1.90160361]]), 13]

In [149]:
C @ ans[0] - d

array([[0.],
       [0.],
       [0.],
       [0.]])

# Обращение симметрично положительно определенной матрицы методом квадратного корня 

In [248]:
A = np.array([[0.1399, 0.5513, 0.9627, 1.3741],
             [0.5513, 10.8634, 15.7276, 20.5918],
             [0.9627, 15.7276, 53.5043, 66.8627],
             [1.3741, 20.5918, 66.8627, 149.3963]])

In [308]:
A = np.array([[1, 1, 1], [1, 2, 2], [1, 2, 3]])

In [309]:
def L(A):
    L = np.ndarray(A.shape)
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            if i == j:
                buf = A[i][i]
                for k in range(i):
                    buf -= L[i][k] ** 2
                L[i][i] = buf ** 0.5
            elif j < i:
                buf = A[i][j] / L[j][j]
                for k in range(j):
                    buf -= L[i][k] * L[j][k] / L[j][j]
                L[i][j] = buf
            else:
                L[i][j] = 0
    return L

In [338]:
def inverse_matrix(A):
    l = L(A)
    P = np.zeros(A.shape)
    n = A.shape[0]
    for i in range(n):
        P[i][i] = 1 / l[i][i]
        for j in range(i+1, n):
            buf = 0
            for k in range(j):
                print(j, k)
                buf += l[j][k] * P[k][i]
            P[i][j] = - buf / l[j][j]

    return P

In [339]:
ans = inverse_matrix(A.copy())

1 0
2 0
2 1
2 0
2 1


In [325]:
A @ ans

array([[-1.,  3.,  2.],
       [-3.,  6.,  4.],
       [-4.,  7.,  6.]])

In [329]:
ans

array([[ 1., -1., -1.],
       [ 0.,  1., -0.],
       [ 0.,  0.,  1.]])

In [330]:
l = L(A)
l

array([[1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.]])

In [333]:
ans @ l

array([[-1., -2., -1.],
       [ 1.,  1.,  0.],
       [ 1.,  1.,  1.]])

In [332]:
np.linalg.inv(l)

array([[ 1.,  0.,  0.],
       [-1.,  1.,  0.],
       [ 0., -1.,  1.]])